Notebook Settings
=================

``` ipython
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%run ../../../notebooks/setup.py
%matplotlib inline
%config InlineBackend.figure_format = 'png'
```

Imports
=======

``` ipython
import torch
import torch.nn as nn
import torch.optim as optim
import torchmetrics
from torch.utils.data import Dataset, TensorDataset, DataLoader

REPO_ROOT = "/home/leon/models/NeuroFlame"

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pal = sns.color_palette("tab10")
DEVICE = 'cuda:0'
```

``` ipython
import sys
sys.path.insert(0, '../../../')

from notebooks.setup import *

import pandas as pd
import torch.nn as nn
from time import perf_counter
from scipy.stats import circmean

from src.network import Network

from src.decode import decode_bump, circcvl

from src.train.dual.task_score import DualScore
```

Helpers
=======

Data Split
----------

``` ipython
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

def split_data(X, Y, train_perc=0.8, batch_size=32):

    if Y.ndim==3:
      X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                          train_size=train_perc,
                                                          stratify=Y[:, 0, 0].cpu().numpy(),
                                                          shuffle=True)
    else:
      X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                          train_size=train_perc,
                                                          stratify=Y[:, 0].cpu().numpy(),
                                                          shuffle=True)

    print(X_train.shape, X_test.shape)
    print(Y_train.shape, Y_test.shape)

    train_dataset = TensorDataset(X_train, Y_train)
    val_dataset = TensorDataset(X_test, Y_test)

    # Create data loaders
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader
```

``` ipython
from sklearn.model_selection import KFold, StratifiedKFold
from torch.utils.data import Dataset, DataLoader, TensorDataset

def cross_val_data(X, Y, n_splits=5, batch_size=32):
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True)

    for fold, (train_index, val_index) in enumerate(kf.split(X, Y[:, 0].cpu().numpy() if Y.ndim == 2 else Y[:, 0, 0].cpu().numpy())):
        X_train, X_val = X[train_index], X[val_index]
        Y_train, Y_val = Y[train_index], Y[val_index]

        print(f'Fold {fold}')
        print("Train:", X_train.shape, Y_train.shape)
        print("Val:", X_val.shape, Y_val.shape)

        train_dataset = TensorDataset(X_train, Y_train)
        val_dataset = TensorDataset(X_val, Y_val)

        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

        yield train_loader, val_loader

# Example use case:
# for train_loader, val_loader in cross_val_data(ff_input, labels, n_splits=5, batch_size=batch_size):
#     train_model(train_loader, val_loader) # Implement your training process here
```

Optimization
------------

``` ipython
def torch_angle_AB(U, V):
      # Calculate the dot product
      dot_product = torch.dot(U, V)

      # Calculate the magnitudes of U and V
      magnitude_U = torch.linalg.norm(U)
      magnitude_V = torch.linalg.norm(V)

      # Compute the cosine of the angle
      cos_theta = dot_product / (magnitude_U * magnitude_V + .00001)

      # Calculate the angle in radians, then convert to degrees
      angle_radians = torch.acos(cos_theta)
      return torch.round(torch.rad2deg(angle_radians))
```

``` ipython
import torch

def training_step(dataloader, model, loss_fn, optimizer, zero_grad=None):

      model.train()
      total_loss = 0.0
      total_batches = len(dataloader)

      for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(model.device), y.to(model.device)

            optimizer.zero_grad()

            model(X)
            y_pred = model.readout
            loss = loss_fn(y_pred, y)

            loss.backward()

            if zero_grad is not None:
                  try:
                        if zero_grad == 'all':
                              model.low_rank.U.grad[:, :] = 0
                              model.low_rank.V.grad[:, :] = 0
                        else:
                              model.low_rank.U.grad[:, zero_grad] = 0
                              model.low_rank.V.grad[:, zero_grad] = 0
                  except:
                        pass

            optimizer.step()

            total_loss += loss.item()

      avg_loss = total_loss / total_batches

      return avg_loss
```

``` ipython
def validation_step(dataloader, model, loss_fn):
    size = len(dataloader.dataset)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(model.device), y.to(model.device)

            model(X)
            y_pred = model.readout

            batch_loss = loss_fn(y_pred, y)
            val_loss += batch_loss.item() * X.size(0)

    val_loss /= size
    return val_loss
```

``` ipython
def optimization(model, train_loader, val_loader, loss_fn, optimizer, num_epochs=100, thresh=0.15, zero_grad=None):
      scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
      # scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.1, verbose=True)
      # scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

      loss_list = []
      val_loss_list = []

      for epoch in range(num_epochs):
          loss = training_step(train_loader, model, loss_fn, optimizer, zero_grad=zero_grad)
          val_loss = validation_step(val_loader, model, loss_fn)

          scheduler.step(val_loss)
          loss_list.append(loss)
          val_loss_list.append(val_loss)

          print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {loss:.4f}, Validation Loss: {val_loss:.4f}')

          if val_loss < thresh and loss < thresh:
              print(f'Stopping training as loss has fallen below the threshold: {loss}, {val_loss}')
              break

          if val_loss > 300:
              print(f'Stopping training as loss is too high: {val_loss}')
              break

          if torch.isnan(torch.tensor(loss)):
              print(f'Stopping training as loss is NaN.')
              break

      return loss_list, val_loss_list
```

Loss
----

``` ipython
import torch
import torch.nn as nn
import torch.nn.functional as F

def safe_mean(tensor):
    """Returns mean or zero if tensor is empty."""
    if tensor.numel() == 0:
        return torch.tensor(0.0, device=tensor.device, dtype=tensor.dtype)
    else:
        return tensor.mean()

class BCEOneClassLoss(nn.Module):
    # Your original BCEOneClassLoss goes here
    def __init__(self):
        super().__init__()
        self.criterion = nn.BCEWithLogitsLoss(reduction='none')

    # def forward(self, logits, targets, class_bal=1):
    #     bce = self.criterion(logits, targets.float())

    #     if class_bal == 0:
    #         mask_pos = (targets == 1)
    #         mask_neg = (targets == 0)
    #         pos_loss = safe_mean(bce[mask_pos])
    #         # For negatives: encourage sigmoid(logit) => 0.5 via MSE
    #         if mask_neg.any():
    #             neg_loss = safe_mean((torch.sigmoid(logits[mask_neg]) - 0.5) ** 2)
    #             return pos_loss + 0.1 * neg_loss
    #         else:
    #             return pos_loss

    #     return safe_mean(bce)

    def forward(self, logits, targets, class_bal=1):
        bce = self.criterion(logits, targets.float())

        if class_bal == 0:
            mask1 = (targets == 1)
            pos_loss = safe_mean(bce[mask1])
            mask0 = (targets == 0)
            # Encourage proba = 0.5 for class 0 (logit=0)
            neutral_loss = safe_mean(self.criterion(logits[mask0], torch.full_like(logits[mask0], 0.5)))
            return pos_loss + 0.01 * neutral_loss

        return safe_mean(bce)

    # def forward(self, logits, targets, class_bal=1):
    #     criterion = nn.BCEWithLogitsLoss(reduction='none')
    #     bce = criterion(logits, targets.float())

    #     if class_bal == 0:
    #         mask = (targets == 1)
    #         return safe_mean(bce[mask])

    #     return safe_mean(bce)

class SignBCELoss(nn.Module):
    def __init__(self, alpha=0.5, thresh=1.0, class_bal=0):
        super().__init__()
        self.alpha = alpha
        self.thresh = thresh
        self.class_bal = class_bal
        self.bce_with_logits = BCEOneClassLoss()

    def forward(self, readout, targets):
        # BCE loss (can be 0 if alpha==1)
        bce_loss = 0.0
        if self.alpha != 1.0:
            bce_loss = self.bce_with_logits(readout, targets, self.class_bal)

        sign_overlap = torch.sign(2 * targets - 1) * readout
        sign_loss = torch.zeros_like(sign_overlap)

        if self.alpha!=0:
            if self.class_bal == 0:
                # Penalize class 0 (targets==0) with |overlap|
                mask0 = (targets == 0)
                if mask0.sum() > 0:
                    sign_loss[mask0] = 0.1 * torch.abs(sign_overlap[mask0])
                    # Penalize class 1 (targets==1) with relu(thresh - overlap)
                mask1 = (targets == 1)
                if mask1.sum() > 0:
                    sign_loss[mask1] = F.relu(self.thresh - sign_overlap[mask1])
            else:
                sign_loss = F.relu(self.thresh - sign_overlap)

        # Combine safely
        loss = ((1 - self.alpha) * bce_loss + self.alpha * safe_mean(sign_loss))

        return loss
```

``` ipython
class AccuracyLoss(nn.Module):
    def __init__(self, thresh=4.0, class_bal=0):
        super(AccuracyLoss, self).__init__()
        self.thresh = thresh
        self.class_bal = class_bal

    def forward(self, readout, targets):
        prob = torch.sigmoid(readout[...])

        mask = (targets[:, 0] == 0).unsqueeze(-1)
        mask = (targets == 1)

        criterion = nn.BCEWithLogitsLoss(reduction='none')
        loss_per_elem = criterion(logits, targets.float())

        if self.class_bal:
            loss = loss_per_elem[mask]
            # loss = torch.where(mask, -torch.log(1 - prob + 1e-8), -torch.log(prob + 1e-8)).mean()
            loss = torch.where(mask, prob**2, (1 - prob)**2).mean()
        else:
            # loss = torch.where(mask, -torch.log(1 - torch.abs(2 * prob - 1) + 1e-8), -torch.log(prob + 1e-8)).mean()
            loss = torch.where(mask, (prob - 0.5)**2, (1 - prob)**2).mean()

        return loss
```

``` ipython
import torch
import torch.nn as nn

class DualLoss(nn.Module):
      def __init__(self, alpha=1.0, thresh=5.0, stim_idx=[], gng_idx=[], cue_idx=[], rwd_idx=-1, zero_idx=[], read_idx=[-1], class_bal=[0]):
            super(DualLoss, self).__init__()
            self.alpha = alpha
            self.thresh = thresh
            self.class_bal = class_bal

            # BL idx
            self.zero_idx = zero_idx
            # Sample idx
            self.stim_idx = torch.tensor(stim_idx, dtype=torch.int, device=DEVICE)
            # Go NoGo
            self.gng_idx= torch.tensor(gng_idx, dtype=torch.int, device=DEVICE)
            # rwd idx for DRT
            self.cue_idx = torch.tensor(cue_idx, dtype=torch.int, device=DEVICE)
            # rwd idx for DPA
            self.rwd_idx = torch.tensor(rwd_idx, dtype=torch.int, device=DEVICE)

            # readout idx
            self.read_idx = read_idx

            self.loss = SignBCELoss(self.alpha, self.thresh)
            self.l1loss = nn.SmoothL1Loss()


      def forward(self, readout, targets):

            zeros = torch.zeros_like(readout[:, self.zero_idx, 0])
            # custom zeros for readout
            loss = self.l1loss(readout[:, self.zero_idx, self.read_idx[0]], zeros)
            # zero memory only before stim
            if len(self.read_idx)>1:
                  loss += self.l1loss(readout[:, :self.stim_idx[0]-1, self.read_idx[1]], zeros[:, :self.stim_idx[0]-1])

            is_stim = (self.stim_idx.numel() != 0)
            is_gng = (self.gng_idx.numel() != 0)
            is_cue = (self.cue_idx.numel() != 0)
            is_rwd = (self.rwd_idx.numel() != 0)

            if is_cue:
                  self.loss.class_bal = self.class_bal[3]
                  loss += self.loss(readout[:, self.cue_idx, self.read_idx[3]], targets[:, 2, :self.cue_idx.shape[0]])

            if is_gng:
                  self.loss.class_bal = 1
                  loss += self.loss(readout[:,  self.gng_idx, self.read_idx[2]], targets[:, 2, :self.gng_idx.shape[0]])

            if is_stim:
                  self.loss.class_bal = 1
                  loss += self.loss(readout[:,  self.stim_idx, self.read_idx[1]], targets[:, 1, :self.stim_idx.shape[0]])

            if is_rwd:
                  self.loss.class_bal = self.class_bal[0]
                  loss += self.loss(readout[:,  self.rwd_idx, self.read_idx[0]], targets[:, 0, :self.rwd_idx.shape[0]])

            return loss
```

Other
-----

``` ipython
def calculate_mean_accuracy_and_sem(accuracies):
    mean_accuracy = accuracies.mean()
    std_dev = accuracies.std(unbiased=True).item()
    sem = std_dev / np.sqrt(len(accuracies))
    return mean_accuracy, sem
```

``` ipython
def angle_AB(A, B):
      A_norm = A / (np.linalg.norm(A) + 1e-5)
      B_norm = B / (np.linalg.norm(B) + 1e-5)

      return int(np.arccos(A_norm @ B_norm) * 180 / np.pi)
```

``` ipython
def get_theta(a, b, GM=0, IF_NORM=0):

      u, v = a, b

      if GM:
          v = b - np.dot(b, a) / np.dot(a, a) * a

      if IF_NORM:
          u = a / np.linalg.norm(a)
          v = b / np.linalg.norm(b)

      return np.arctan2(v, u) % (2.0 * np.pi)
```

``` ipython
def get_idx(model, rank=1):
      # print(model.low_rank.U.shape)
      # ksi = torch.vstack((model.low_rank.U[:,0], model.low_rank.U[:,1]))
      ksi = torch.hstack((model.low_rank.V, model.low_rank.U)).T
      ksi = ksi[:, :model.Na[0]]

      try:
            readout = model.low_rank.linear.weight.data
            ksi = torch.vstack((ksi, readout))
      except:
            pass

      print('ksi', ksi.shape)

      ksi = ksi.cpu().detach().numpy()
      theta = get_theta(ksi[0], ksi[rank])

      return theta.argsort()
```

``` ipython
def get_overlap(model, rates):
      ksi = model.odors.cpu().detach().numpy()
      return rates @ ksi.T / rates.shape[-1]
```

``` ipython
import scipy.stats as stats

def plot_smooth(data, ax, color):
      mean = data.mean(axis=0)
      ci = smooth.std(axis=0, ddof=1) * 1.96

      # Plot
      ax.plot(mean, color=color)
      ax.fill_between(range(data.shape[1]), mean - ci, mean + ci, alpha=0.25, color=color)

```

``` ipython
def convert_seconds(seconds):
      h = seconds // 3600
      m = (seconds % 3600) // 60
      s = seconds % 60
      return h, m, s
```

plots
-----

``` ipython
import pickle as pkl
import os
def pkl_save(obj, name, path="."):
    os.makedirs(path, exist_ok=True)
    destination = path + "/" + name + ".pkl"
    print("saving to", destination)
    pkl.dump(obj, open(destination, "wb"))


def pkl_load(name, path="."):
    source = path + "/" + name + '.pkl'
    # print('loading from', source)
    return pkl.load(open( source, "rb"))

```

``` ipython
def add_vlines(ax=None, mouse=""):
    t_BL = [0, 1]
    t_STIM = [1 , 2]
    t_ED = [2, 3]
    t_DIST = [3 , 4]
    t_MD = [4 , 5]
    t_CUE = [5 , 5.5]
    t_RWD = [5.5, 6.0]
    t_LD = [6.0 , 7.0]
    t_TEST = [7.0, 8.0]
    t_RWD2 = [11 , 12]

    # time_periods = [t_STIM, t_DIST, t_TEST, t_CUE, t_RWD, t_RWD2]
    # colors = ["b", "b", "b", "g", "y", "y"]

    time_periods = [t_STIM, t_DIST, t_TEST, t_CUE]
    colors = ["b", "b", "b", "g"]

    if ax is None:
        for period, color in zip(time_periods, colors):
            plt.axvspan(period[0], period[1], alpha=0.1, color=color)
    else:
        for period, color in zip(time_periods, colors):
            ax.axvspan(period[0], period[1], alpha=0.1, color=color)

```

``` ipython
def plot_rates_selec(rates, idx, thresh=0.5, figname='fig.svg'):
        ordered = rates[..., idx]
        fig, ax = plt.subplots(1, 2, figsize=[2*width, height])
        r_max = thresh * np.max(rates[0])

        ax[0].imshow(rates[0].T, aspect='auto', cmap='jet', vmin=0, vmax=r_max)
        ax[0].set_ylabel('Neuron #')
        ax[0].set_xlabel('Step')

        ax[1].imshow(ordered[0].T, aspect='auto', cmap='jet', vmin=0, vmax=r_max)
        ax[1].set_yticks(np.linspace(0, model.Na[0].cpu().detach(), 5), np.linspace(0, 360, 5).astype(int))
        ax[1].set_ylabel('Pref. Location (°)')
        ax[1].set_xlabel('Step')
        plt.savefig(figname, dpi=300)
        plt.show()
```

``` ipython
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import sem, t

def mean_ci(data):
  # Calculate mean and SEM
  mean = np.nanmean(data, axis=0)
  serr = sem(data, axis=0, nan_policy='omit')

  # Calculate the t critical value for 95% CI
  n = np.sum(~np.isnan(data), axis=0)
  t_val = t.ppf(0.975, df=n - 1)  # 0.975 for two-tailed 95% CI

  # Calculate 95% confidence intervals
  ci = t_val * serr

  return mean, ci

def plot_overlap_dpa(readout, y, axis=0, label=['pair', 'unpair'], figname='fig.svg', title='first'):

    fig, ax = plt.subplots(1, 2, figsize=[2*width, height], sharey=True)

    time = np.linspace(0, 10, readout.shape[1])
    trial = [0, 1]
    colors = ['r', 'b', 'g']
    ls = ['--', '-']

    for j in range(2): # pair unpair
        for i in range(2):
            # if axis == 0:
            #     sign_readout = 2*y[-1, :, np.newaxis, np.newaxis] * readout
            #     data = sign_readout[(y[0]==i) & (y[1]==trial[j]), :, axis]
            # else:
            data = readout[(y[0]==i) & (y[1]==trial[j]), :, axis]

            mean, ci = mean_ci(data)
            ax[j].plot(time, mean, ls=ls[i], label=label[i], color=colors[j])
            ax[j].fill_between(time, mean - ci, mean + ci, color=colors[j], alpha=0.1)

        add_vlines(ax[j])
        ax[j].set_xlabel('Time (s)')

        if axis==0:
            ax[j].set_ylabel('A/B Overlap (Hz)')
        elif axis==1:
            ax[j].set_ylabel('GNG Overlap (Hz)')
        else:
            ax[j].set_ylabel('Readout (Hz)')

        ax[j].axhline(0, color='k', ls='--')

    plt.savefig('../figures/dual/%s' % figname, dpi=300)
    plt.show()
```

``` ipython
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import sem, t

def mean_ci(data):
  # Calculate mean and SEM
  mean = np.nanmean(data, axis=0)
  serr = sem(data, axis=0, nan_policy='omit')

  # Calculate the t critical value for 95% CI
  n = np.sum(~np.isnan(data), axis=0)
  t_val = t.ppf(0.975, df=n - 1)  # 0.975 for two-tailed 95% CI

  # Calculate 95% confidence intervals
  ci = t_val * serr

  return mean, ci
```

``` ipython
def plot_avg_overlap(readout, n_batch, labels=['A', 'B'], figname='fig.svg'):
      fig, ax = plt.subplots(1, 2, figsize=[2*width, height])

      time = np.linspace(0, 10, readout.shape[1])
      size = readout.shape[0] // 2
      print(readout.shape[0], size)

      readout = readout.reshape((3, ))

      for i in range(readout.shape[-1]):
            if i==0:
                  ax[i].plot(time, (readout[:size, :, i].T - readout[size:,:,i].T), ls='-', label=labels[0])
            else:
                  ax[i].plot(time, readout[size:, :, i].T, ls='--', label='Go')

            add_vlines(ax[i])
            ax[i].set_xlabel('Time (s)')

      ax[0].set_ylabel('Sample Overlap (Hz)')
      ax[1].set_ylabel('Go/NoGo Overlap (Hz)')
      # ax[2].set_ylabel('Readout (Hz)')

      # plt.legend(fontsize=10, frameon=False)
      plt.savefig(figname, dpi=300)
      plt.show()
```

``` ipython
def plot_m0_m1_phi(rates, idx, figname='fig.svg'):

      m0, m1, phi = decode_bump(rates[..., idx], axis=-1)
      fig, ax = plt.subplots(1, 3, figsize=[3*width, height])

      time = np.linspace(0, 10, m0.T.shape[0])

      ax[0].plot(time, m0[:2].T)
      ax[0].plot(time, m0[2:].T, '--')
      #ax[0].set_ylim([0, 360])
      #ax[0].set_yticks([0, 90, 180, 270, 360])
      ax[0].set_ylabel('$\mathcal{F}_0$ (Hz)')
      ax[0].set_ylabel('Activity (Hz)')
      ax[0].set_xlabel('Time (s)')
      add_vlines(ax[0])

      ax[1].plot(time, m1[:2].T)
      ax[1].plot(time, m1[2:].T, '--')
      # ax[1].set_ylim([0, 360])
      # ax[1].set_yticks([0, 90, 180, 270, 360])
      ax[1].set_ylabel('$\mathcal{F}_1$ (Hz)')
      ax[1].set_ylabel('Bump Amplitude (Hz)')
      ax[1].set_xlabel('Time (s)')
      add_vlines(ax[1])

      ax[2].plot(time, phi[:2].T * 180 / np.pi)
      ax[2].plot(time, phi[2:].T * 180 / np.pi, '--')
      ax[2].set_ylim([0, 360])
      ax[2].set_yticks([0, 90, 180, 270, 360])
      ax[2].set_ylabel('Bump Center (°)')
      ax[2].set_xlabel('Time (s)')
      add_vlines(ax[2])

      plt.savefig(figname, dpi=300)
      plt.show()
```

``` ipython
from matplotlib.patches import Circle

def plot_fix_points(rates, ax, title='', color='k'):
    m0, m1, phi = decode_bump(rates[:, -1], axis=-1)

    x = np.cos(phi)
    y = np.sin(phi)

    xNoGo = np.cos(3*np.pi /2.)
    yNoGo = np.sin(3*np.pi /2)

    xGo = np.cos(np.pi /2.)
    yGo = np.sin(np.pi /2)

    # rad = np.max(np.sqrt(x**2+y**2))

    ax.plot(x, y, 'o', ms=15, color=color)
    ax.plot(xGo, yGo, 'o', ms=15, color='w', markeredgecolor='k')
    ax.plot(xNoGo, yNoGo, 'o', ms=15, color='w', markeredgecolor='k')
    circle = Circle((0., 0.), 1, fill=False, edgecolor='k')
    ax.add_patch(circle)

    # Set the aspect of the plot to equal to make the circle circular
    ax.set_aspect('equal')
    ax.set_title(title)
    ax.axis('off')
    # plt.savefig('fp_dpa.svg', dpi=300)
    # plt.show()
```

``` ipython
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Define custom colormap with red at the center
cdict = {
    'red':   [(0.0, 0.0, 0.0),
              (0.5, 1.0, 1.0),
              (1.0, 1.0, 1.0)],
    'green': [(0.0, 0.0, 0.0),
              (0.5, 0.0, 0.0),
              (1.0, 1.0, 1.0)],
    'blue':  [(0.0, 1.0, 1.0),
              (0.5, 0.0, 0.0),
              (1.0, 0.0, 0.0)]
}

custom_cmap = LinearSegmentedColormap('RedCenterMap', cdict)

# Plot to visualize the colormap
gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

fig, ax = plt.subplots(figsize=(6, 1))
ax.imshow(gradient, aspect='auto', cmap=custom_cmap)
ax.set_axis_off()
plt.show()
```

``` ipython
def plot_overlap(readout, labels=['pair', 'unpair'], figname='fig.svg'):
      fig, ax = plt.subplots(1, readout.shape[-1], figsize=[readout.shape[-1]*width, height])

      time = np.linspace(0, 10, readout.shape[1])
      size = readout.shape[0] // 2

      for i in range(readout.shape[-1]):
            ax[i].plot(time, readout[:size, :, i].T, ls='-', label=labels[0])
            if i==0:
                  ax[i].plot(time, -readout[size:, :, i].T, ls='--', label=labels[1])
            else:
                  ax[i].plot(time, readout[size:, :, i].T, ls='--', label=labels[1])

            add_vlines(ax[i])
            ax[i].set_xlabel('Time (s)')

      ax[0].set_ylabel('Sample Overlap (Hz)')
      ax[1].set_ylabel('Go/NoGo Overlap (Hz)')
      if readout.shape[-1] == 3:
            ax[-1].set_ylabel('Readout (Hz)')

      # ax[1].legend(fontsize=10, frameon=False)
      plt.savefig(figname, dpi=300)
      plt.show()
```

Model
=====

``` ipython
kwargs = {}
```

``` ipython
REPO_ROOT = "/home/leon/models/NeuroFlame"

conf_name = "train_dual.yml"
DEVICE = 'cuda:0'

# seed = np.random.randint(0, 1e6)
seed = 1
print(seed)
```

``` ipython
model = Network(conf_name, REPO_ROOT, VERBOSE=0, DEVICE=DEVICE, SEED=seed, N_BATCH=1, **kwargs)
device = torch.device(DEVICE if torch.cuda.is_available() else 'cpu')
model.to(device);
```

``` ipython
batch_size = 16
learning_rate = 0.1
bce_alpha = 1.0
thresh = 2.0
```

``` ipython
import gc
gc.collect()

torch.cuda.empty_cache()
torch.cuda.device(DEVICE)   # where X is the GPU index, e.g., 0, 1
torch.cuda.synchronize()
torch.cuda.reset_accumulated_memory_stats(DEVICE)
```

DPA Task
========

Training
--------

### Parameters

``` ipython
model.J_STP.requires_grad = True

if model.LR_READOUT:
    for param in model.low_rank.linear.parameters():
        param.requires_grad = False
    model.low_rank.linear.bias.requires_grad = False
```

``` ipython
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.shape)
```

``` ipython
steps = np.arange(0, model.N_STEPS - model.N_STEADY, model.N_WINDOW)
mask = steps >= (model.N_STIM_ON[-1].cpu().numpy() - model.N_STEADY)
rwd_idx = np.where(mask)[0]
print('rwd', rwd_idx)

# mask for A/B memory from sample to test
stim_mask = (steps >= (model.N_STIM_ON[0].cpu().numpy() - model.N_STEADY)) & (steps < (model.N_STIM_ON[-1].cpu().numpy() - model.N_STEADY))
stim_idx = np.where(stim_mask)[0]
print('stim', stim_idx)

model.lr_eval_win = np.max((rwd_idx.shape[0], stim_idx.shape[0]))

mask_zero = ~mask  # & ~stim_mask
zero_idx = np.where(mask_zero)[0]
print('zero', zero_idx)
```

### Inputs and Labels

``` ipython
N_BATCH = 256
model.N_BATCH = N_BATCH

model.lr_eval_win = np.max( (rwd_idx.shape[0], stim_idx.shape[0]))

ff_input = []
labels = np.zeros((2, 4, model.N_BATCH, model.lr_eval_win))

l=0
for i in [-1, 1]:
        for k in [-1, 1]:

            model.I0[0] = i # sample
            model.I0[4] = k # test

            if i == 1:
                    labels[1, l] = np.ones((model.N_BATCH, model.lr_eval_win))

            if i==k: # Pair Trials
                labels[0, l] = np.ones((model.N_BATCH, model.lr_eval_win))

            l+=1

            ff_input.append(model.init_ff_input())

labels = torch.tensor(labels, dtype=torch.float, device=DEVICE).reshape(2, -1, model.lr_eval_win).transpose(0, 1)

ff_input = torch.vstack(ff_input)
print('ff_input', ff_input.shape, 'labels', labels.shape)
```

### Run

``` ipython
splits = [split_data(ff_input, labels, train_perc=0.8, batch_size=batch_size)]
# splits = cross_val_data(ff_input, labels, n_splits=5, batch_size=batch_size)
criterion = DualLoss(alpha=bce_alpha, thresh=thresh, rwd_idx=rwd_idx, stim_idx=stim_idx, zero_idx=zero_idx, class_bal=[1.0, 1.0], read_idx=[1, 0])

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# or add l2 regularization
# params = [{'params': [model.low_rank.U, model.low_rank.V], 'weight_decay': .01}]
# optimizer = optim.Adam(params, lr=learning_rate)
```

``` ipython
print('training DPA')
num_epochs = 10

start = perf_counter()

for train_loader, val_loader in splits:
    loss, val_loss = optimization(model, train_loader, val_loader, criterion, optimizer, num_epochs, zero_grad=None)
end = perf_counter()

print("Elapsed (with compilation) = %dh %dm %ds" % convert_seconds(end - start))

torch.save(model.state_dict(), '%s/dpa_%d.pth' % (model.SAVE_PATH, seed))
```

``` ipython
print(model.J_STP)
```

Testing
-------

``` ipython
model_state_dict = torch.load('%s/dpa_%d.pth' % (model.SAVE_PATH, seed))
model.load_state_dict(model_state_dict)
model.eval();
```

``` ipython
N_BATCH = 1
model.N_BATCH = N_BATCH

ff_input = []

# 2 readouts (sample/choice), 4 conditions AC, AD, BC, BD
labels = np.zeros((2, 4, model.N_BATCH))

l=0
for i in [-1, 1]:
        for k in [-1, 1]:

            model.I0[0] = i # sample
            model.I0[4] = k # test

            if i == 1:
                    labels[1, l] = np.ones(model.N_BATCH)

            if i==k: # Pair Trials
                labels[0, l] = np.ones(model.N_BATCH)

            l+=1
            with torch.no_grad():
                    ff_input.append(model.init_ff_input())

labels = torch.tensor(labels, dtype=torch.float, device=DEVICE).reshape(2, -1)

ff_input = torch.vstack(ff_input)
print('ff_input', ff_input.shape, 'labels', labels.shape)
```

``` ipython
with torch.no_grad():
    rates = model.forward(ff_input=ff_input).detach().cpu().numpy()
print(rates.shape)
```

``` ipython
def plot_overlap_label(readout, y, task=0, label=['pair', 'unpair'], figname='fig.svg', title='first'):
    '''
    y[0] is pairs, y[1] is samples, y[2] is task if not None
    '''
    print(y.shape)
    size = y.shape[0]
    if size ==2:
        ones_slice = np.zeros(y.shape)
        y_ = np.vstack((y.copy(), ones_slice))
        task = 0
    else:
        y_ = y.copy()

    print(y_.shape)
    fig, ax = plt.subplots(1, size, figsize=[size*width, height], sharey=True)

    time = np.linspace(0, 10, readout.shape[1])
    colors = ['r', 'b', 'g']
    ls = ['--', '-', '-.', ':']
    label = ['AD', 'AC', 'BD', 'BC']

    for k in range(2): # readout
        for j in range(2): # sample
            for i in range(2): # pair
                data = readout[(y_[0]==i) & (y_[1]==j) & (y_[2]==task), :, k]

                mean, ci = mean_ci(data)

                ax[k].plot(time, mean, ls=ls[i+2*j], label=label[i+2*j], color=colors[task], alpha=1-j/4)
                ax[k].fill_between(time, mean - ci, mean + ci, color=colors[task], alpha=0.1)

                add_vlines(ax[k])
                ax[k].set_xlabel('Time (s)')

        if k==0:
            ax[k].set_ylabel('A/B Overlap (Hz)')
        elif k==1:
            ax[k].set_ylabel('GNG Overlap (Hz)')
        else:
            ax[k].set_ylabel('Readout (Hz)')

        ax[k].axhline(0, color='k', ls='--')

    plt.legend(fontsize=10)
    plt.savefig('../figures/dual/%s' % figname, dpi=300)
    plt.show()
```

``` ipython
plot_overlap_label(model.readout.cpu().detach().numpy(), labels.cpu().numpy(), task=0)
```

``` ipython
U = model.low_rank.U.cpu().detach().numpy()
V = model.low_rank.V.cpu().detach().numpy()

fig, ax = plt.subplots(1, 2, figsize=[2*width, height])
# ax[0].hist(U[:, 0], histtype='step', bins='auto')
# ax[0].hist(U[:, 1], histtype='step', bins='auto')
ax[0].hist(V[:, 0], histtype='step', bins='auto')
ax[1].hist(V[:, 1], histtype='step', bins='auto')
ax[0].set_xlabel('$ n_{AB} $')
ax[1].set_xlabel('$ n_{GNG} $')

ax[0].set_ylabel('Count')
ax[1].set_ylabel('Count')
plt.show()
```

``` ipython
odors = model.odors.cpu().numpy()

m1 = U[:, 0]
n1 = V[:, 0]

m2 = U[:, 1]
n2 = V[:, 1]

vectors = [m1, n1, m2, n2]
labels = ['$m_\\text{AB}$', '$n_\\text{AB}$', '$m_\\text{GnG}$', '$n_\\text{GnG}$']
```

``` ipython
import numpy as np
import matplotlib.pyplot as plt

# Calculate the covariance matrix
num_vectors = len(vectors)
cov_matrix = np.zeros((num_vectors, num_vectors))

for i in range(num_vectors):
    for j in range(num_vectors):
        cov_matrix[i][j] = angle_AB(vectors[i], vectors[j])

# Mask the upper triangle
mask = np.triu(np.ones_like(cov_matrix, dtype=bool))
masked_cov_matrix = np.ma.masked_array(cov_matrix, mask=mask)
```

``` ipython
fig, ax = plt.subplots(figsize=(8, 6))

# Plot the masked covariance matrix
img = plt.imshow(masked_cov_matrix, cmap='coolwarm', interpolation='nearest')
# cbar = plt.colorbar(label='Angle (°)')
# cbar.set_ticks([30, 90, 120])

# Set axis labels on top and left
plt.xticks(ticks=np.arange(num_vectors), labels=labels)
plt.yticks(ticks=np.arange(num_vectors), labels=labels)

# Invert y-axis
ax.xaxis.set_ticks_position('top')
ax.xaxis.set_label_position('top')

# ax.yaxis.set_ticks_position('right')
# ax.yaxis.set_label_position('right')
ax.invert_yaxis()

for i in range(num_vectors):
    for j in range(i + 1):
        plt.text(j, i, f'{cov_matrix[i, j]:.0f}', ha='center', va='center', color='black')

plt.savefig('../figures/dual/cov_dpa_%d.svg' % seed, dpi=300)
plt.show()
```

``` ipython
```

Go/NoGo Task
============

``` ipython
model_state_dict = torch.load('%s/dpa_%d.pth' % (model.SAVE_PATH, seed))
model.load_state_dict(model_state_dict)
```

Training
--------

``` ipython
model.J_STP.requires_grad = False
```

``` ipython
for name, param in model.named_parameters():
      if param.requires_grad:
            print(name, param.shape)
```

``` ipython
steps = np.arange(0, model.N_STEPS - model.N_STEADY, model.N_WINDOW)

# mask for lick/nolick  from cue to test
rwd_mask = (steps >= (model.N_STIM_ON[2].cpu().numpy() - model.N_STEADY)) & (steps < (model.N_STIM_ON[4].cpu().numpy() - model.N_STEADY))
rwd_idx = np.where(rwd_mask)[0]
print('rwd', rwd_idx)

# mask for Go/NoGo memory from dist to cue
stim_mask = (steps >= (model.N_STIM_ON[1].cpu().numpy() - model.N_STEADY)) & (steps < (model.N_STIM_ON[2].cpu().numpy() - model.N_STEADY))
stim_idx = np.where(stim_mask)[0]
# stim_idx = []
print('stim', stim_idx)

mask_zero = (steps < (model.N_STIM_ON[1].cpu().numpy() - model.N_STEADY))
zero_idx = np.where(mask_zero)[0]
print('zero', zero_idx)

model.lr_eval_win = np.max( (rwd_idx.shape[0], stim_idx.shape[0]))
```

``` ipython
model.N_BATCH = 512

model.I0[0] = 0
model.I0[1] = 1
model.I0[2] = 1
model.I0[3] = 0
model.I0[4] = 0

Go = model.init_ff_input()

model.I0[0] = 0
model.I0[1] = -1
model.I0[2] = 1
model.I0[3] = 0
model.I0[4] = 0

NoGo = model.init_ff_input()

ff_input = torch.cat((Go, NoGo))
print(ff_input.shape)
```

``` ipython
labels_Go = torch.ones((model.N_BATCH, model.lr_eval_win))
labels_NoGo = torch.zeros((model.N_BATCH, model.lr_eval_win))
labels = torch.cat((labels_Go, labels_NoGo))
print(labels.shape)
labels =  labels.repeat((2, 1, 1))
labels = torch.transpose(labels, 0, 1)
print('labels', labels.shape)
```

### Run

``` ipython
train_loader, val_loader = split_data(ff_input, labels, train_perc=0.8, batch_size=batch_size)
criterion = DualLoss(alpha=bce_alpha, thresh=thresh, rwd_idx=rwd_idx, zero_idx=zero_idx, stim_idx=stim_idx, class_bal=[0.0, 1.0], read_idx=[1, 1])
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
```

``` ipython
print('training DRT')
num_epochs = 5

start = perf_counter()
loss, val_loss = optimization(model, train_loader, val_loader, criterion, optimizer, num_epochs, zero_grad=0)
end = perf_counter()

print("Elapsed (with compilation) = %dh %dm %ds" % convert_seconds(end - start))
```

``` ipython
torch.save(model.state_dict(), '%s/dual_naive_%d.pth' % (model.SAVE_PATH, seed))
```

Test
----

``` ipython
model_state_dict = torch.load('%s/dual_naive_%d.pth' % (model.SAVE_PATH, seed))
model.load_state_dict(model_state_dict)
model.eval();
```

``` ipython
model.N_BATCH = 1

model.I0[0] = 0
model.I0[1] = 1
model.I0[2] = 1
model.I0[3] = 0.0
model.I0[4] = 0.0

A = model.init_ff_input()

model.I0[0] = 0 # NoGo
model.I0[1] = -1 # cue
model.I0[2] = 1 # rwd
model.I0[3] = 0.0
model.I0[4] = 0.0

B = model.init_ff_input()

ff_input = torch.cat((A, B))
print('ff_input', ff_input.shape)
```

``` example
672340e2-1a3a-44e8-97f7-1bce6b5ea086
```

``` ipython
with torch.no_grad():
    rates = model.forward(ff_input=ff_input).cpu().detach().numpy()
print(rates.shape)
```

``` ipython
plot_overlap(model.readout.cpu().detach().numpy(), labels=['Go', 'NoGo'], figname='../figures/dual/GoNoGo_overlaps_%d.svg' % seed)
```

``` ipython
U = model.low_rank.U.cpu().detach().numpy()
V = model.low_rank.V.cpu().detach().numpy()

fig, ax = plt.subplots(1, 2, figsize=[2*width, height])
# ax[0].hist(U[:, 0], histtype='step', bins='auto')
# ax[0].hist(U[:, 1], histtype='step', bins='auto')
ax[0].hist(V[:, 0], histtype='step', bins='auto')
ax[1].hist(V[:, 1], histtype='step', bins='auto')
ax[0].set_xlabel('$ n_{AB} $')
ax[1].set_xlabel('$ n_{GNG} $')

ax[0].set_ylabel('Count')
ax[1].set_ylabel('Count')
plt.show()
```

``` ipython
U = model.low_rank.U.cpu().detach().numpy()[model.slices[0], 0]
V = model.low_rank.V.cpu().detach().numpy()[model.slices[0], 0]

odors = model.odors.cpu().numpy()

m = model.low_rank.U.cpu().detach().numpy()[model.slices[0], 1]
n = model.low_rank.V.cpu().detach().numpy()[model.slices[0], 1]

vectors = [U, V, m, n]
labels = ['$m_\\text{AB}$', '$n_\\text{AB}$', '$m_\\text{GnG}$', '$n_\\text{GnG}$']

import numpy as np
import matplotlib.pyplot as plt

# Calculate the covariance matrix
num_vectors = len(vectors)
cov_matrix = np.zeros((num_vectors, num_vectors))

for i in range(num_vectors):
    for j in range(num_vectors):
        cov_matrix[i][j] = angle_AB(vectors[i], vectors[j])

# Mask the upper triangle
mask = np.triu(np.ones_like(cov_matrix, dtype=bool))
masked_cov_matrix = np.ma.masked_array(cov_matrix, mask=mask)

plt.figure(figsize=(8, 6))

# Plot the masked covariance matrix
img = plt.imshow(masked_cov_matrix, cmap=custom_cmap, interpolation='nearest', vmin=30, vmax=150)
cbar = plt.colorbar(label='Angle (°)')
cbar.set_ticks([30, 90, 120])

# Set axis labels on top and left
# plt.gca().xaxis.tick_top()
plt.xticks(ticks=np.arange(num_vectors), labels=labels)
plt.yticks(ticks=np.arange(num_vectors), labels=labels)

# Invert y-axis
plt.gca().invert_yaxis()

for i in range(num_vectors):
    for j in range(i + 1):
        plt.text(j, i, f'{cov_matrix[i, j]:.0f}', ha='center', va='center', color='black')

plt.savefig('../figures/dual/cov_drt_%d.svg' % seed, dpi=300)
plt.show()
```

``` ipython
```

Dual Task
=========

Testing
-------

### Simulations

``` ipython
model_state_dict = torch.load('%s/dual_naive_%d.pth' % (model.SAVE_PATH, seed))
model.load_state_dict(model_state_dict)
model.eval();
```

``` ipython
steps = np.arange(0, model.N_STEPS - model.N_STEADY, model.N_WINDOW)

mask_rwd = (steps >= (model.N_STIM_ON[-1].cpu().numpy() - model.N_STEADY))
rwd_idx = np.where(mask_rwd)[0]
print('rwd', rwd_idx)

mask_cue = (steps >= (model.N_STIM_ON[2].cpu().numpy() - model.N_STEADY)) & (steps <= (model.N_STIM_OFF[3].cpu().numpy() - model.N_STEADY))
cue_idx = np.where(mask_cue)[0]
print('cue', cue_idx)

mask_GnG = (steps >= (model.N_STIM_OFF[1].cpu().numpy() - model.N_STEADY)) & (steps <= (model.N_STIM_ON[2].cpu().numpy() - model.N_STEADY))
gng_idx = np.where(mask_GnG)[0]
print('GnG', gng_idx)

mask_stim = (steps >= (model.N_STIM_ON[0].cpu().numpy() - model.N_STEADY)) & (steps <= (model.N_STIM_ON[-1].cpu().numpy() - model.N_STEADY))
stim_idx = np.where(mask_stim)[0]
print('stim', stim_idx)

mask_zero = ~mask_rwd & ~mask_cue & ~mask_stim
zero_idx = np.where(mask_zero)[0]
print('zero', zero_idx)
```

``` ipython
U = model.low_rank.U.cpu().detach().numpy()[model.slices[0], 0]
V = model.low_rank.V.cpu().detach().numpy()[model.slices[0], 0]

odors = model.odors.cpu().numpy()

m = model.low_rank.U.cpu().detach().numpy()[model.slices[0], 1]
n = model.low_rank.V.cpu().detach().numpy()[model.slices[0], 1]

vectors = [U, V, m, n]
labels = ['$m_\\text{AB}$', '$n_\\text{AB}$', '$m_\\text{GnG}$', '$n_\\text{GnG}$']

import numpy as np
import matplotlib.pyplot as plt

# Calculate the covariance matrix
num_vectors = len(vectors)
cov_matrix = np.zeros((num_vectors, num_vectors))

for i in range(num_vectors):
    for j in range(num_vectors):
        cov_matrix[i][j] = angle_AB(vectors[i], vectors[j])

# Mask the upper triangle
mask = np.triu(np.ones_like(cov_matrix, dtype=bool))
masked_cov_matrix = np.ma.masked_array(cov_matrix, mask=mask)

#+RESULTS:

plt.figure(figsize=(8, 6))

# Plot the masked covariance matrix
img = plt.imshow(masked_cov_matrix, cmap=custom_cmap, interpolation='nearest', vmin=30, vmax=150)
cbar = plt.colorbar(label='Angle (°)')
cbar.set_ticks([30, 90, 120])

# Set axis labels on top and left
# plt.gca().xaxis.tick_top()
plt.xticks(ticks=np.arange(num_vectors), labels=labels)
plt.yticks(ticks=np.arange(num_vectors), labels=labels)

# Invert y-axis
plt.gca().invert_yaxis()

for i in range(num_vectors):
    for j in range(i + 1):
        plt.text(j, i, f'{cov_matrix[i, j]:.0f}', ha='center', va='center', color='black')

plt.savefig('../figures/dual/cov_naive_%d.svg' % seed, dpi=300)
plt.show()
```

``` ipython
N_BATCH = 10
model.N_BATCH = N_BATCH

model.lr_eval_win = np.max( (rwd_idx.shape[0], cue_idx.shape[0]))

ff_input = []
labels = np.zeros((3, 12, model.N_BATCH, model.lr_eval_win))

l=0
for j in [0, 1, -1]:
    for i in [-1, 1]:
        for k in [-1, 1]:

            model.I0[0] = i # sample
            if i==1:
                labels[1, l] = i * np.ones((model.N_BATCH, model.lr_eval_win))

            model.I0[1] = j # distractor
            model.I0[4] = k # test

            if i==k: # Pair Trials
                labels[0, l] = np.ones((model.N_BATCH, model.lr_eval_win))

            if j==1: # Go
                model.I0[2] = 1 # cue
                model.I0[3] = 0 * model.IF_RL # rwd

                labels[2, l] = np.ones((model.N_BATCH, model.lr_eval_win))
            elif j==-1: # NoGo
                model.I0[2] = 1 # cue
                model.I0[3] = 0.0 # rwd
                labels[2, l] = -np.ones((model.N_BATCH, model.lr_eval_win))
            else: # DPA
                model.I0[2] = 0 # cue
                model.I0[3] = 0 # rwd

            l+=1

            ff_input.append(model.init_ff_input())


labels = torch.tensor(labels, dtype=torch.float, device=DEVICE).reshape(3, -1, model.lr_eval_win).transpose(0, 1)

ff_input = torch.vstack(ff_input)
print('ff_input', ff_input.shape, 'labels', labels.shape)
```

``` ipython
with torch.no_grad():
    rates = model.forward(ff_input=ff_input).cpu().detach().numpy()
print(rates.shape)
```

### Overlaps

``` ipython
readout = model.readout.cpu().detach().numpy()
y_labels = labels[..., 0].T.cpu().numpy().copy()
print(readout.shape, y_labels.shape)
```

``` ipython
def plot_overlap_label(readout, y, task=0, label=['pair', 'unpair'], figname='fig.svg', title='first'):
    '''
    y[0] is pairs, y[1] is samples, y[2] is task if not None
    '''
    print(y.shape)
    size = y.shape[0]
    if size ==2:
        ones_slice = np.zeros(y.shape)
        y_ = np.vstack((y.copy(), ones_slice))
        task = 0
    else:
        y_ = y.copy()
        tasks = [0, 1, -1]

    print(y_.shape)
    fig, ax = plt.subplots(1, 2, figsize=[2*width, height], sharey=True)

    time = np.linspace(0, 10, readout.shape[1])
    colors = ['r', 'b', 'g']
    ls = ['--', '-', '-.', ':']
    label = ['AD', 'AC', 'BD', 'BC']

    for k in range(2): # readout
        for j in range(2): # sample
            for i in range(2): # pair
                data = readout[(y_[0]==i) & (y_[1]==j) & (y_[2]==task), :, k]
                mean, ci = mean_ci(data)

                ax[k].plot(time, mean, ls=ls[i+2*j], label=label[i+2*j], color=colors[task], alpha=1-j/4)
                ax[k].fill_between(time, mean - ci, mean + ci, color=colors[task], alpha=0.1)

        add_vlines(ax[k])
        ax[k].set_xlabel('Time (s)')

        if k==0:
            ax[k].set_ylabel('A/B Overlap (Hz)')
        elif k==1:
            ax[k].set_ylabel('GNG Overlap (Hz)')
        else:
            ax[k].set_ylabel('Readout (Hz)')

        ax[k].axhline(0, color='k', ls='--')

    plt.legend(fontsize=10)
    plt.savefig('../figures/dual/%s' % figname, dpi=300)
    plt.show()
```

``` ipython
plot_overlap_label(readout, y_labels, task=0, figname='sample_overlaps_naive.svg')
```

``` ipython
plot_overlap_label(readout, y_labels, task=1, figname='sample_overlaps_naive.svg')
```

``` ipython
plot_overlap_label(readout, y_labels, task=-1, figname='sample_overlaps_naive.svg')
```

``` ipython
```

### Perf

``` ipython
criterion = DualScore(cue_idx=cue_idx, rwd_idx=rwd_idx, read_idx=[1, 1])
dpa_perf, drt_perf = criterion(model.readout, labels.clone())
```

``` ipython
dpa_mean = []
dpa_sem = []
for i in [0, 1, -1]:
        y = torch.where(labels[:, 2, 0]==i)

        mean_, sem_ = calculate_mean_accuracy_and_sem(dpa_perf[y])
        dpa_mean.append(mean_)
        dpa_sem.append(sem_)

dpa_mean = torch.stack(dpa_mean)
dpa_sem = np.stack(dpa_sem)
```

``` ipython
drt_mean, drt_sem = calculate_mean_accuracy_and_sem(drt_perf)
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[1.5*width, height])

pal = ['r', 'b', 'g']
for i in range(3):
    ax[0].errorbar(i, dpa_mean[i].item(), yerr=dpa_sem[i].item(), fmt='o', label='Naive',
                   color=pal[i], ecolor=pal[i], elinewidth=3, capsize=5)

ax[0].set_xlim(-1, 4)
# ax[0].set_ylim(0.4, 1.1)

ax[0].set_ylabel('DPA Accuracy')
ax[0].set_xticks([1], ['Naive'])
ax[0].axhline(y=0.5, color='k', linestyle='--')

ax[1].errorbar(0, drt_mean.item(), yerr=drt_sem.item(), fmt='o', label='Naive',
             color='k', ecolor='k', elinewidth=3, capsize=5)

ax[1].set_xlim(-1, 2)
ax[1].set_ylim(0.4, 1.1)

ax[1].set_ylabel('Go/NoGo Accuracy')
ax[1].set_xticks([0, 1], ['Naive', 'Expert'])
ax[1].axhline(y=0.5, color='k', linestyle='--')

plt.savefig('../figures/dual/dual_perf_%d.svg' % seed, dpi=300)

plt.show()
```

``` ipython
```

Training
--------

``` ipython
steps = np.arange(0, model.N_STEPS - model.N_STEADY, model.N_WINDOW)

mask_rwd = (steps >= (model.N_STIM_ON[-1].cpu().numpy() - model.N_STEADY))
rwd_idx = np.where(mask_rwd)[0]
print('rwd', rwd_idx)

mask_cue = (steps >= (model.N_STIM_ON[2].cpu().numpy() - model.N_STEADY)) & (steps <= (model.N_STIM_OFF[3].cpu().numpy() - model.N_STEADY))
cue_idx = np.where(mask_cue)[0]
print('cue', cue_idx)

mask_GnG = (steps >= (model.N_STIM_ON[1].cpu().numpy() - model.N_STEADY)) & (steps <= (model.N_STIM_ON[2].cpu().numpy() - model.N_STEADY))
gng_idx = np.where(mask_GnG)[0]
print('GnG', gng_idx)

mask_stim = (steps >= (model.N_STIM_ON[0].cpu().numpy() - model.N_STEADY)) & (steps <= (model.N_STIM_ON[-1].cpu().numpy() - model.N_STEADY))
stim_idx = np.where(mask_stim)[0]
print('stim', stim_idx)

mask_zero = ~mask_rwd & ~mask_cue & ~mask_stim
zero_idx = np.where(mask_zero)[0]
print('zero', zero_idx)
```

``` ipython
model.N_BATCH = 64

model.lr_eval_win = np.max( (rwd_idx.shape[0], cue_idx.shape[0], stim_idx.shape[0], gng_idx.shape[0]))

ff_input = []
labels = np.zeros((3, 12, model.N_BATCH, model.lr_eval_win))

l=0
for i in [-1, 1]:
    for j in [-1, 0, 1]:
        for k in [-1, 1]:

            model.I0[0] = i # sample
            model.I0[1] = j # distractor
            model.I0[4] = k # test

            if i==1:
                labels[1, l] = np.ones((model.N_BATCH, model.lr_eval_win))

            if i==k: # Pair Trials
                labels[0, l] = np.ones((model.N_BATCH, model.lr_eval_win))

            if j==1: # Go
                model.I0[2] = float(1) # cue
                model.I0[3] = 0 * model.IF_RL # rwd

                labels[2, l] = np.ones((model.N_BATCH, model.lr_eval_win))

            elif j==-1: # NoGo
                model.I0[2] = 1 # cue
                model.I0[3] = 0.0 # rwd

            else: # DPA
                model.I0[2] = 0 # cue
                model.I0[3] = 0 # rwd

            l+=1

            ff_input.append(model.init_ff_input())

labels = torch.tensor(labels, dtype=torch.float, device=DEVICE).reshape(3, -1, model.lr_eval_win).transpose(0, 1)
ff_input = torch.vstack(ff_input)
print('ff_input', ff_input.shape, 'labels', labels.shape)
```

``` ipython
train_loader, val_loader = split_data(ff_input, labels, train_perc=0.8, batch_size=batch_size)

criterion = DualLoss(alpha=bce_alpha, thresh=thresh,
                     stim_idx=stim_idx, gng_idx=gng_idx,
                     cue_idx=cue_idx, rwd_idx=rwd_idx, zero_idx=zero_idx,
                     class_bal=[1.0, 0.0, 1.0, 0.0], read_idx=[1, 0, 1, 1])

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
```

``` ipython
print('training Dual')
num_epochs = 5
start = perf_counter()

loss, val_loss = optimization(model, train_loader, val_loader, criterion, optimizer, num_epochs, zero_grad=None)
end = perf_counter()
print("Elapsed (with compilation) = %dh %dm %ds" % convert_seconds(end - start))
```

``` ipython
torch.save(model.state_dict(), '%s/dual_train_%d.pth' % (model.SAVE_PATH, seed))
```

``` ipython
```

Re-Testing
----------

### run

``` ipython
model_state_dict = torch.load('%s/dual_train_%d.pth' % (model.SAVE_PATH, seed))
model.load_state_dict(model_state_dict)
model.eval();
```

``` ipython
steps = np.arange(0, model.N_STEPS - model.N_STEADY, model.N_WINDOW)

mask_rwd = (steps >= (model.N_STIM_ON[-1].cpu().numpy() - model.N_STEADY))
rwd_idx = np.where(mask_rwd)[0]
print('rwd', rwd_idx)

mask_cue = (steps >= (model.N_STIM_ON[2].cpu().numpy() - model.N_STEADY)) & (steps <= (model.N_STIM_OFF[3].cpu().numpy() - model.N_STEADY))
cue_idx = np.where(mask_cue)[0]
print('cue', cue_idx)

mask_GnG = (steps >= (model.N_STIM_OFF[1].cpu().numpy() - model.N_STEADY)) & (steps <= (model.N_STIM_ON[2].cpu().numpy() - model.N_STEADY))
gng_idx = np.where(mask_GnG)[0]
print('GnG', gng_idx)

mask_stim = (steps >= (model.N_STIM_ON[0].cpu().numpy() - model.N_STEADY)) & (steps <= (model.N_STIM_ON[-1].cpu().numpy() - model.N_STEADY))
stim_idx = np.where(mask_stim)[0]
print('stim', stim_idx)

mask_zero = ~mask_rwd & ~mask_cue & ~mask_stim
zero_idx = np.where(mask_zero)[0]
print('zero', zero_idx)
```

``` ipython
U = model.low_rank.U.cpu().detach().numpy()[model.slices[0], 0]
V = model.low_rank.V.cpu().detach().numpy()[model.slices[0], 0]

odors = model.odors.cpu().numpy()

m = model.low_rank.U.cpu().detach().numpy()[model.slices[0], 1]
n = model.low_rank.V.cpu().detach().numpy()[model.slices[0], 1]

vectors = [U, V, m, n]
labels = ['$m_\\text{AB}$', '$n_\\text{AB}$', '$m_\\text{GnG}$', '$n_\\text{GnG}$']

import numpy as np
import matplotlib.pyplot as plt

# Calculate the covariance matrix
num_vectors = len(vectors)
cov_matrix = np.zeros((num_vectors, num_vectors))

for i in range(num_vectors):
    for j in range(num_vectors):
        cov_matrix[i][j] = angle_AB(vectors[i], vectors[j])

# Mask the upper triangle
mask = np.triu(np.ones_like(cov_matrix, dtype=bool))
masked_cov_matrix = np.ma.masked_array(cov_matrix, mask=mask)

plt.figure(figsize=(8, 6))

# Plot the masked covariance matrix
img = plt.imshow(masked_cov_matrix, cmap=custom_cmap, interpolation='nearest', vmin=30, vmax=150)
cbar = plt.colorbar(label='Angle (°)')
cbar.set_ticks([30, 90, 120])

# Set axis labels on top and left
# plt.gca().xaxis.tick_top()
plt.xticks(ticks=np.arange(num_vectors), labels=labels)
plt.yticks(ticks=np.arange(num_vectors), labels=labels)

# Invert y-axis
plt.gca().invert_yaxis()

for i in range(num_vectors):
    for j in range(i + 1):
        plt.text(j, i, f'{cov_matrix[i, j]:.0f}', ha='center', va='center', color='black')
plt.savefig('../figures/dual/cov_train_%d.svg' % seed, dpi=300)
plt.show()
```

``` ipython
N_BATCH = 10
model.N_BATCH = N_BATCH

model.lr_eval_win = np.max( (rwd_idx.shape[0], cue_idx.shape[0]))

ff_input = []
labels = np.zeros((3, 12, model.N_BATCH, model.lr_eval_win))

l=0
for j in [0, 1, -1]:
    for i in [-1, 1]:
        for k in [-1, 1]:

            model.I0[0] = i # sample
            if i==1:
                labels[1, l] = i * np.ones((model.N_BATCH, model.lr_eval_win))

            model.I0[1] = j # distractor
            model.I0[4] = k # test

            if i==k: # Pair Trials
                labels[0, l] = np.ones((model.N_BATCH, model.lr_eval_win))

            if j==1: # Go
                model.I0[2] = float(1) # cue
                model.I0[3] = 0 * model.IF_RL # rwd

                labels[2, l] = np.ones((model.N_BATCH, model.lr_eval_win))
            elif j==-1: # NoGo
                model.I0[2] = float(1) # cue
                model.I0[3] = 0.0 # rwd
                labels[2, l] = -np.ones((model.N_BATCH, model.lr_eval_win))
            else: # DPA
                model.I0[2] = 0 # cue
                model.I0[3] = 0 # rwd

            l+=1

            ff_input.append(model.init_ff_input())


labels = torch.tensor(labels, dtype=torch.float, device=DEVICE).reshape(3, -1, model.lr_eval_win).transpose(0, 1)

ff_input = torch.vstack(ff_input)
print('ff_input', ff_input.shape, 'labels', labels.shape)
```

``` ipython
with torch.no_grad():
    rates = model.forward(ff_input=ff_input).cpu().detach().numpy()
print(rates.shape)
```

### Overlaps

``` ipython
readout = model.readout.cpu().detach().numpy()
y_labels = labels[..., 0].T.cpu().numpy().copy()
print(readout.shape, y_labels.shape)
```

``` ipython
def plot_overlap_label(readout, y, task=0, label=['pair', 'unpair'], figname='fig.svg', title='first'):
    '''
    y[0] is pairs, y[1] is samples, y[2] is task if not None
    '''
    print(y.shape)
    size = y.shape[0]
    if size ==2:
        ones_slice = np.zeros(y.shape)
        y_ = np.vstack((y.copy(), ones_slice))
        task = 0
    else:
        y_ = y.copy()
        tasks = [0, 1, -1]

    print(y_.shape)
    fig, ax = plt.subplots(1, 2, figsize=[2*width, height], sharey=True)

    time = np.linspace(0, 10, readout.shape[1])
    colors = ['r', 'b', 'g']
    ls = ['--', '-', '-.', ':']
    label = ['AD', 'AC', 'BD', 'BC']

    for k in range(2): # readout
        for j in range(2): # sample
            for i in range(2): # pair
                data = readout[(y_[0]==i) & (y_[1]==j) & (y_[2]==task), :, k]
                mean, ci = mean_ci(data)

                ax[k].plot(time, mean, ls=ls[i+2*j], label=label[i+2*j], color=colors[task], alpha=1-j/4)
                ax[k].fill_between(time, mean - ci, mean + ci, color=colors[task], alpha=0.1)

        add_vlines(ax[k])
        ax[k].set_xlabel('Time (s)')

        if k==0:
            ax[k].set_ylabel('A/B Overlap (Hz)')
        elif k==1:
            ax[k].set_ylabel('GNG Overlap (Hz)')
        else:
            ax[k].set_ylabel('Readout (Hz)')

        ax[k].axhline(0, color='k', ls='--')

    plt.legend(fontsize=10)
    plt.savefig('../figures/dual/%s' % figname, dpi=300)
    plt.show()
```

``` ipython
plot_overlap_label(readout, y_labels, task=0, figname='sample_overlaps_naive.svg')
```

``` ipython
plot_overlap_label(readout, y_labels, task=1, figname='sample_overlaps_naive.svg')
```

``` ipython
plot_overlap_label(readout, y_labels, task=-1, figname='sample_overlaps_naive.svg')
```

``` ipython
```

### Perf

``` ipython
criterion = DualScore(cue_idx=cue_idx, rwd_idx=rwd_idx, read_idx=[1, 1])
dpa_perf, drt_perf = criterion(model.readout, labels.clone())
```

``` ipython
dpa_mean2 = []
dpa_sem2 = []
for i in [0, 1, -1]:
        y = torch.where(labels[:, 2, 0]==i)

        mean_, sem_ = calculate_mean_accuracy_and_sem(dpa_perf[y])
        dpa_mean2.append(mean_)
        dpa_sem2.append(sem_)

dpa_mean2 = torch.stack(dpa_mean2)
dpa_sem2 = np.stack(dpa_sem2)
```

``` ipython
drt_mean2, drt_sem2 = calculate_mean_accuracy_and_sem(drt_perf)
```

``` ipython
fig, ax = plt.subplots(1, 2, figsize=[1.5*width, height])

pal = ['r', 'b', 'g']
for i in range(3):
    ax[0].errorbar(i, dpa_mean[i].item(), yerr=dpa_sem[i].item(), fmt='o', label='Naive',
                   color=pal[i], ecolor=pal[i], elinewidth=3, capsize=5)

    ax[0].errorbar(i+4, dpa_mean2[i].item(), yerr=dpa_sem2[i].item(), fmt='o', label='Naive',
                   color=pal[i], ecolor=pal[i], elinewidth=3, capsize=5)

ax[0].set_xlim(-1, 7)
ax[0].set_ylim(0.4, 1.1)

ax[0].set_ylabel('DPA Accuracy')
ax[0].set_xticks([1, 5], ['Naive', 'Expert'])
ax[0].axhline(y=0.5, color='k', linestyle='--')

ax[1].errorbar(0, drt_mean.item(), yerr=drt_sem.item(), fmt='o', label='Naive',
             color='k', ecolor='k', elinewidth=3, capsize=5)
ax[1].errorbar(1, drt_mean2.item(), yerr=drt_sem2.item(), fmt='o', label='Naive',
             color='k', ecolor='k', elinewidth=3, capsize=5)

ax[1].set_xlim(-1, 2)
ax[1].set_ylim(0.4, 1.1)

ax[1].set_ylabel('Go/NoGo Accuracy')
ax[1].set_xticks([0, 1], ['Naive', 'Expert'])
ax[1].axhline(y=0.5, color='k', linestyle='--')

plt.savefig('../figures/dual/dual_perf_%d.svg' % seed, dpi=300)

plt.show()
```

``` ipython
U = model.low_rank.U.cpu().detach().numpy()
V = model.low_rank.V.cpu().detach().numpy()

fig, ax = plt.subplots(1, 2, figsize=[2*width, height])
# ax[0].hist(U[:, 0], histtype='step', bins='auto')
# ax[0].hist(U[:, 1], histtype='step', bins='auto')
ax[0].hist(V[:, 0], histtype='step', bins='auto')
ax[1].hist(V[:, 1], histtype='step', bins='auto')
ax[0].set_xlabel('$ n_{AB} $')
ax[1].set_xlabel('$ n_{GNG} $')

ax[0].set_ylabel('Count')
ax[1].set_ylabel('Count')
plt.show()
```

``` ipython
plt.scatter(V[:, 0], V[:, 1])
plt.xlabel('$ n_{AB} $')
plt.ylabel('$ n_{GNG} $')
plt.show()
```

``` ipython
```